In [35]:
import re
import os
from pathlib import Path

In [41]:
mat_re = re.compile(
    r'''^((?:[^/]*/)*)?         # Path
        ((?:[@+]\w+\.)*)       # Namespace
        (\w+)(?:\.\w+)?$       # Filename
        ''', re.VERBOSE)

mat_re_namespace = re.compile(
    r'([@+]\w+)\.', re.VERBOSE  # Namespace list
)

def parse_autodoc_input(arg:str, matlab_root_dir:str, src_path:str):

    (path, namespacePath, filename) = mat_re.match(arg).groups()
    namespace = mat_re_namespace.findall(namespacePath)

    basepath = Path(matlab_root_dir if matlab_root_dir else src_path)
    abspath = (basepath / Path(path or '.')).resolve()
    filepath = (abspath / '/'.join(namespace)).resolve()

    return (filepath, filename, abspath, namespace)


In [45]:
arg = '../@package.+main.test.m'
matlab_root_dir = 'C:/Users'
src_path = os.getcwd()

parse_autodoc_input(arg, matlab_root_dir, src_path)

(WindowsPath('C:/@package/+main'),
 'test',
 WindowsPath('C:/'),
 ['@package', '+main'])